In [1]:
#import necessary libraries and get the cards
import json5
with open('cah-base-en.deck.json5', encoding='utf-8') as fh:
    d = json5.load(fh)

# process black cards
# simpy mush them together and replace all dicts with a single underscore
bc = d['calls']
flatten = lambda t:[item for sublist in t for item in sublist]
bc = ["".join([x if type(x) == str else '_' for x in flatten(d['calls'][i])]) for i in range(len(d['calls']))]
bc = [i.replace("__", "_") for i in bc]
# grab white cards
wc = d['responses']
print(bc[:10])
nbc = [i for i in bc if i.count('_') <= 2]

['Hey Reddit! I’m _.Ask me anything.', 'Introducing X-treme Baseball!It’s like baseball, but with _!', 'What is Batman’s guilty pleasure?_.', 'TSA guidelines now prohibit _ on airplanes.', 'Next from J.K. Rowling: _.', 'That’s right, I killed _.How, you ask?_.', 'I’m sorry professor, but I couldn’t complete my homework because of _.', 'And the Academy Award for _ goes to _.', 'Dude, do not go in that bathroom.There’s _ in there.', 'How did I lose my virginity?_.']


In [3]:
#import libraries
from sentence_transformers import SentenceTransformer
s_model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [4]:
# create all possible card combinations
# just brute force it!

import itertools
def make_card_combs(wcs, bcs):
    onec = wcs
    twoc = [i for i in itertools.permutations(wcs, 2)]
    threec = [i for i in itertools.permutations(wcs, 3)]
    bcs = [i.split("_") for i in bcs]
    plays = []

    len_one_slot = 0
    len_two_slot = 0
    len_three_slot = 0

    for bc in bcs:
        bc = [i.strip() for i in bc]
        if len(bc) == 2:
            len_one_slot +=1 
            for p in onec:
               s = bc[0] + " " + p + " " +  bc[1]
               s.replace(". ", " .")
               plays.append(s)
        
        elif len(bc) == 3:
            len_two_slot +=1
            for p in twoc:
                s = bc[0] + " " + p[0] + " " +  bc[1] + " " + p[1] + bc[2]
                s.replace(". ", " .")
                plays.append(s)
        elif len(bc) == 4:
            len_three_slot +=1
            #for p in threec:
            #   s = bc[0] + " " + p[0] + " " +  bc[1] + " " + p[1] + bc[2] + " " + p[2] + " " + bc[3]
            #   s.replace(". ", " .")
            #   plays.append(s)
        
    print("one_slot:", len_one_slot)
    print("two_slot:", len_two_slot)
    print("three_slot:", len_three_slot)
    
    return [i for i in plays if i != None]

In [5]:
#call make all cards
all_card_combs = make_card_combs(wc, nbc)
print(len(all_card_combs))

one_slot: 66
two_slot: 12
three_slot: 0
3027000


In [6]:
#generate from sbert
from tqdm import tqdm
embeddings = {}
for c in tqdm(all_card_combs):
    embeddings[c] = s_model.encode(c)

100%|██████████| 3027000/3027000 [22:05:38<00:00, 38.06it/s]      


In [7]:
#save the embeddings
import pickle
with open('embed_dict_bert_2cards_768d_raw.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [ ]:
#convert to embedding matrix
import numpy as np
ordered_cards = list(embeddings.keys())
ordered_embeddings = [embeddings[i].tolist()  for i in ordered_cards]
#print(type(ordered_embeddings), len(ordered_embeddings))
#print(type(ordered_embeddings[0]), len(ordered_embeddings[0]))
ordered_embeddings = np.array(ordered_embeddings)
#print(ordered_embeddings, ordered_embeddings.shape)